In [1]:
%pip install xarray[complete]
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd
import xarray as xr
import s3fs




In [3]:
bucket = "open-mars"
file_name = "mars-4d-data/openmars_my28-32_part_0.nc"


In [4]:

s3 = boto3.client('s3') 
# 's3' is a key word. create connection to S3 using default config and all buckets within S3

obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket


# initial_df = pd.read_csv() # 'Body' is a key word

In [41]:
%%time
fs_s3 = s3fs.S3FileSystem(anon=False)
file_list = fs_s3.ls(f"{bucket}/mars-4d-data/")

# s3_file_obj = fs_s3.open('s3://open-mars/mars-4d-data/openmars_my28-32_part_74.nc', mode='rb')

CPU times: user 639 µs, sys: 159 µs, total: 798 µs
Wall time: 676 µs


In [42]:
print(file_list)

['open-mars/mars-4d-data/', 'open-mars/mars-4d-data/corrupt-files', 'open-mars/mars-4d-data/openmars_my28-32_part_0.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_1.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_10.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_11.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_12.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_13.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_14.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_15.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_16.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_17.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_18.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_19.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_2.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_20.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_22.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_24.nc', 'open-mars/mars-4d-data/openmars_my28-32_part_25.nc', 'open-mars/mars-4

In [50]:
%%timeit
nc_files = [file for file in file_list if file.endswith('.nc')]


for nc_file in nc_files:
    s3_file_obj = fs_s3.open(nc_file, mode='rb')
    print(f'Reading file {nc_file}')
    df = xr.open_dataset(s3_file_obj).to_dataframe()
    file_name = nc_file.split('/')[-1].split('.')[0]
    s3_url = f's3://open-mars/mars-4d-data-parquet/{file_name}.parquet'
    print(f'now going to write file {s3_url} to parquet')

    df.to_parquet(s3_url)

Reading file open-mars/mars-4d-data/openmars_my28-32_part_0.nc
now going to write file s3://open-mars/mars-4d-data-parquet/openmars_my28-32_part_0.parquet to parquet
Reading file open-mars/mars-4d-data/openmars_my28-32_part_1.nc
now going to write file s3://open-mars/mars-4d-data-parquet/openmars_my28-32_part_1.parquet to parquet
Reading file open-mars/mars-4d-data/openmars_my28-32_part_10.nc
now going to write file s3://open-mars/mars-4d-data-parquet/openmars_my28-32_part_10.parquet to parquet
Reading file open-mars/mars-4d-data/openmars_my28-32_part_11.nc
now going to write file s3://open-mars/mars-4d-data-parquet/openmars_my28-32_part_11.parquet to parquet
Reading file open-mars/mars-4d-data/openmars_my28-32_part_12.nc
now going to write file s3://open-mars/mars-4d-data-parquet/openmars_my28-32_part_12.parquet to parquet
Reading file open-mars/mars-4d-data/openmars_my28-32_part_13.nc
now going to write file s3://open-mars/mars-4d-data-parquet/openmars_my28-32_part_13.parquet to parq

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [15]:
df.head()

Ls  MY          ps       tsurf  \
lon    lat       lev    time                                                  
-180.0 87.499992 0.9995 2941.083252  124.303047  28  671.463989  213.820465   
                        2941.166748  124.343605  28  669.990723  213.623337   
                        2941.250000  124.384163  28  667.152771  213.089417   
                        2941.333252  124.424728  28  667.891357  212.378693   
                        2941.416748  124.465294  28  666.232239  211.526703   

                                     co2ice   dustcol         u         v  \
lon    lat       lev    time                                                
-180.0 87.499992 0.9995 2941.083252     0.0  0.362743 -1.676928  9.747583   
                        2941.166748     0.0  0.391976  0.262620  6.571910   
                        2941.250000     0.0  0.375872  0.726103  4.269494   
                        2941.333252     0.0  0.347010 -0.460853  1.976945   
                        2941.416748     0.0  0.347146 -1.797833  0.395463   

                                           temp  
lon    lat       lev    time                     
-180.0 87.499992 0.9995 2941.083252  213.373016  
                        2941.166748  213.742737  
                        2941.250000  213.954803  
                        2941.333252  213.715225  
                        2941.416748  212.377243

In [16]:
memory_usage = df.memory_usage(deep=True) / 1024 ** 2
memory_usage.head(10)

Index      155.742352
Ls         124.584961
MY         124.584961
ps         124.584961
tsurf      124.584961
co2ice     124.584961
dustcol    124.584961
u          124.584961
v          124.584961
temp       124.584961
dtype: float64